# Z-критерий для двух долей 

In [3]:
import numpy as np
import pandas as pd

import scipy
import statsmodels
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

## Задача о баннерах

Мы хотим рекламировать некоторую услугу или товар с помощью
рекламного баннера.
Для этого у нас уже есть некоторый стандартный баннер,
но дизайнеры разработали для нас новый, более прекрасный баннер.
Нам с вами хочется проверить, правда ли, что **новый баннер действительно лучше,
чем старый**, и нам имеет смысл заменить старый баннер на новый.
А может быть, это не так, и имеет смысл все-таки остаться со старым.
Для того чтобы это понять, мы можем провести следующий эксперимент.

Создаем простую веб-формочку, в которую размещаем оба этих баннера.
Далее просим некоторое количество людей, например, **1000** человек,
на эти баннеры посмотреть и нажать на кнопочку like, если баннер им понравился.
В результате этого эксперимента мы получим следующую выборку данных:
**последовательность нулей и единиц для каждого баннера**, где ноль будет обозначать
отсутствие клика, то есть ситуацию, когда человеку баннер не понравился,
а единичка — наличие клика, ситуация, когда баннер человеку понравился. 

### Загрузка данных

In [4]:
data = pd.read_csv('banner_click_stat.txt', header = None, sep = '\t')
data.columns = ['banner_a', 'banner_b']

In [5]:
data.head()

,banner_a,banner_b
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0


In [6]:
data.describe()

,banner_a,banner_b
count,1000.000000,1000.000000
mean,0.037000,0.053000
std,0.188856,0.224146
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,1.000000,1.000000


Видно, что доля кликов по баннеру Б больше.

### Интервальные оценки долей

$$\frac1{ 1 + \frac{z^2}{n} } \left( \hat{p} + \frac{z^2}{2n} \pm z \sqrt{ \frac{ \hat{p}\left(1-\hat{p}\right)}{n} + \frac{z^2}{4n^2} } \right), \;\; z \equiv z_{1-\frac{\alpha}{2}}$$ 

In [7]:
conf_interval_banner_a = proportion_confint(sum(data.banner_a), 
                                            data.shape[0],
                                            method = 'wilson')
conf_interval_banner_b = proportion_confint(sum(data.banner_b), 
                                            data.shape[0],
                                            method = 'wilson')

In [8]:
print('95%% confidence interval for a click probability, banner a: [%f, %f]' % conf_interval_banner_a)
print('95%% confidence interval for a click probability, banner b [%f, %f]' % conf_interval_banner_b)

95% confidence interval for a click probability, banner a: [0.026961, 0.050582]
95% confidence interval for a click probability, banner b [0.040747, 0.068675]


Проблема, интервалы пересекаются. Однозначно сделать вывод, какой баннер лучше, нельзя.

Рассчитаем некоторые критерии, которые позволят нам проверить гипотезу о
том, что доли действительно отличаются, что они не равны.

Рассмотрим z-критерий для разности долей. 

### Z-критерий для разности долей (независимые выборки)

  -| $X_1$ | $X_2$
  --------- | -------------|----------
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

$$Z-статистика: Z({X_1, X_2}) =  \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{P(1 - P)(\frac{1}{n_1} + \frac{1}{n_2})}}$$
$$P = \frac{\hat{p}_1{n_1} + \hat{p}_2{n_2}}{{n_1} + {n_2}} $$

 Реализуем функцию для рассчета доверительного интервала:

In [9]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

Реализуем функцию расчета z-статистики:

In [10]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

Реализуем функцию расчёта p-value:

In [11]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

Выведем доверительный интервал на разность двух
долей, видно, что он включает ноль, поэтому о существенном различии пока сказать нельзя.

In [12]:
print("95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_ind(data.banner_a, data.banner_b))

95% confidence interval for a difference between proportions: [-0.034157, 0.002157]


Протестируем двустороннюю альтернативу. Гипотеза того, что доли одинаковые, против гипотезы,
что доли неодинаковые.

In [13]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(data.banner_a, data.banner_b)))

p-value: 0.084379


Видно, что значение p-value больше 0.05. Не можем отвергнуть нулевую гипотезу.

Воспользуемся предположением,
что новый баннер все-таки лучше, чем старый, и проверим гипотезу против альтернативы less:

In [14]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(data.banner_a, data.banner_b), 'less'))

p-value: 0.042189


Можем отвергнуть гипотезу h0 о том, что доли одинаковые, на уровне значимости 95%.

Теперь рассмотрим случай, когда баннеры оценивали одни и те же люди, то есть выборки связанные.

### Z-критерий для разности долей (связанные выборки)

  $X_1$ \ $X_2$ | 1| 0 | $\sum$
  -------- | -------------|----------|----------
  1  | e | f | e + f
  0  | g | h | g + h
  $\sum$ | e + g| f + h | n  
  
$$ \hat{p}_1 = \frac{e + f}{n}$$

$$ \hat{p}_2 = \frac{e + g}{n}$$

$$ \hat{p}_1 - \hat{p}_2 = \frac{f - g}{n}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

$$Z-статистика: Z({X_1, X_2}) = \frac{f - g}{\sqrt{f + g - \frac{(f-g)^2}{n}}}$$

Реализуем формулу для оценки доверительного интервала:

In [15]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

Реализуем формулу для подсчета z-статистики:

In [20]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [17]:
print("95%% confidence interval for a difference between proportions: [%f, %f]" \
      % proportions_diff_confint_rel(data.banner_a, data.banner_b))

95% confidence interval for a difference between proportions: [-0.026689, -0.005311]


Весь доверительный интервал лежит левее нуля.
Это дает нам основание предполагать, что новый баннер действительно лучше,
чем старый. 

Проверим это на двусторонней альтернативе:

In [18]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_rel(data.banner_a, data.banner_b)))

p-value: 0.003349


И на альтернативе less в предположении, что новый баннер лучше:

In [19]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_rel(data.banner_a, data.banner_b), 'less'))

p-value: 0.001675


Видно, что смело можем отвергать нулевую гипотезу, и новый баннер действительно лучше старого.

## Задача из "Разрушителей легенд"

В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

Используем Z-критерий для разности долей (независимые выборки):

  -| $X_1$ | $X_2$
  --------- | -------------|----------
  1  | 10 | 4 
  0  | 24 | 12 
  $\sum$ | $34$| $16$
  
$$ \hat{p}_1 = \frac{10}{34}$$

$$ \hat{p}_2 = \frac{4}{12}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

$$Z-статистика: Z({X_1, X_2}) =  \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{P(1 - P)(\frac{1}{n_1} + \frac{1}{n_2})}}$$
$$P = \frac{\hat{p}_1{n_1} + \hat{p}_2{n_2}}{{n_1} + {n_2}} $$

Нулевая гипотеза состоит в том, что зевота незаразительна.

Альтернативная в том, что заразительна.

In [106]:
a = 10 # зевают
b = 4 # не зевают
n1 = 34
n2 = 16
p1 = a / n1
p2 = b / n2
P = float(p1*n1 + p2*n2) / (n1 + n2)
z_crit = (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))
z_crit

0.32410186177608225

То есть используем альтернативу 'greater':

In [107]:
round(proportions_diff_z_test(z_crit, alternative = 'greater'),4)

0.3729

**Ответ:** судя по данным эксперимента зевота не заразительна.

## Задача про банкноты

Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными.

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

   1. Логистическая регрессия по признакам $X_1,X_2,X_3$;
   2. Логистическая регрессия по признакам $X_4,X_5,X_6$.

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

In [216]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Читаем данные:

In [217]:
data = pd.read_csv('banknotes.txt', sep = '\t')
data.shape

(200, 7)

In [218]:
data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


Делим выборку:

In [219]:
train, test = train_test_split(data, test_size = 50, random_state=1)

In [220]:
X_train1 = train[['X1', 'X2', 'X3']]
X_train2 = train[['X4', 'X5', 'X6']]
y_train = train['real']

X_test1 = test[['X1', 'X2', 'X3']]
X_test2 = test[['X4', 'X5', 'X6']]
y_test = test['real']

Создадим и обучим модели:

In [221]:
model1 = LogisticRegression(solver='liblinear')
model2 = LogisticRegression(solver='liblinear')
m1 = model1.fit(X_train1, y_train)
m2 = model2.fit(X_train2, y_train)

Предскажем результаты на тесте и посчитаем score:

In [222]:
m1_pred = m1.predict(X_test1)
m2_pred = m2.predict(X_test2)
sc1 = accuracy_score(m1_pred, y_test)
sc2 = accuracy_score(m2_pred, y_test)
print('Accuracy первой модели:', sc1)
print('Accuracy второй модели:', sc2)

Accuracy первой модели: 0.8
Accuracy второй модели: 0.98


Видно, что сильно отличаются, значит p-value должно быть маленьким. Выборки зависимые! Посчитаем:

In [223]:
sample1 = list(m1_pred != y_test)
sample2 = list(m2_pred != y_test)

z-статистика:

In [224]:
z_crit = proportions_diff_z_stat_rel(sample1, sample2)
z_crit

2.9386041680175268

p-value:

In [225]:
proportions_diff_z_test(z_crit, alternative = 'two-sided')

0.0032969384555543435

**Ответ:** p-value гораздо меньше нуля, отвергаем нулевую гипотезу.

### В предыдущей задаче посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.

Воспользуемся написанными функциями:

In [228]:
borders = proportions_diff_confint_rel(sample1, sample2, alpha = 0.05)
borders

(0.059945206279614305, 0.3000547937203857)

In [229]:
round(borders[0],4)

0.0599

## Задача про экзамен GMAT

Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки. 

**Нулевая гипотеза** - что средние не отличаются.

**Альтернатива** - более высокий средний балл статистически значим.

Рассчитаем значения z-критерия:

In [148]:
true_mean = 525
std = 100
my_mean = 541.4
n = 100
z_crit = (my_mean-true_mean)/(std/n**0.5)
z_crit

1.6399999999999977

In [150]:
1 - scipy.stats.norm.cdf(z_crit)

0.05050258347410397

**Ответ:** Нулевая гипотеза не отвергается!

Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки. 

In [155]:
true_mean = 525
std = 100
my_mean = 541.5
n = 100
z_crit = (my_mean - true_mean)/(std/n**0.5)
z_crit

1.65

In [158]:
1 - scipy.stats.norm.cdf(z_crit)

0.0494714680336481

**Ответ:** Отвергаем нулевую гипотезу!